In [234]:
from copy import copy
from matplotlib import pyplot as plt
from pathlib import Path
from plotly.offline import iplot, init_notebook_mode
from plotly.graph_objs import Mesh3d
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core.structure import Structure, Lattice
from pymatgen.core.tensors import Tensor, SquareTensor, get_uvec
from pymatgen.core.operations import SymmOp
from abipy.core.structure import Structure as AbiStructure

import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go


In [235]:
# Import sys and set the path of the customized modules
import sys
sys.path.append('/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/Custom_Modules/')

# Import importlib.reload to be able to reload dynamically modified modules
from importlib import reload

# Import custom modules (reloadable when modified)
import SHG_Tensor_Func as shg
shg = reload(shg)

# TODO

DONE
1. Define function to verify that Kleinman symmetry is respected in a given d_ijk, i.e., any arrangement of i,j, and k.
2. Define shape of d_ijk for every point groups, i.e., components=0 and components=other components providing the sign is adapted (sometimes opposite sign)
    - Function returning the correct shape for a given point group
3. Define function to check if a given d_ijk does not respect the conventional shape for a given point group 
    - Done BUT catches some correct dijk which have convergence issues or very small values for example...
4. Define function to find the conventional shape for a given d_ijk and a given point group
    - First, test the usual rotations as was done for Naccarato dataset (script "NLODS_figues.ipynb" in jnb/Articles/01...)
    - Second, if necessary, perform all rotations possible to find the one which minimizes the sum of the absolute values of the components that are expected to be 0. Refine iteratively maybe? Because pretty computationally expensive otherwise.
    - Third, try different combination of 180 degrees rotations around all the axes to make the largest component in absolute value take the positive sign.
5. Correct all tensors of my dataset and add this step to the new d_ijk computed.

STILL TO DO

6. Try using MatTen to predict the full SHG tensor now that it adopts the conventional form.

email to bilbao:

- By Kleinman symmetry, there are only 10 independent components. If we look at the SHG tensor under Kleinman symmetry for the point group 1, we see that there are indeed 10 independent components, but it is written that, for one independent component, some of its "clones" might be of opposite sign. Maybe I am missing something, but I would say that, while the intrinsic symmetry of the crystal can force certain independent components to be zero or can force two independent components to be equal in absolute value but opposite in sign, the intrinsic symmetry of the crystal cannot force two "clones" of an independent component to be opposite since they should be strictly equal by Kleinman symmetry. I verified this statement on my own SHG data which inherently respects Kleinman symmetry. Am I missing something that might explain two "clones" of an independent component to be opposite?

# Functions

In [236]:
def verify_kleinman(d):
    # Work with tensor in non-Voigt form
    dijk = shg.from_voigt(d)


    # Kleinman implies that all arrangements of i,j, and k are equivalent
    klmn_equival = ["ijk", "kij", "jki", "ikj", "jik", "kji"]
    # Meaning there are only 10 independent components 
    # The -1 at the end is there to treat those combination as array indices
    klmn_indpt = np.array([ [1,1,1], [1,2,2], [1,3,3], [1,2,3], [1,1,3], [1,1,2],
                            [2,2,2], [2,3,3], [2,2,3],
                            [3,3,3]])-1
    
    # Iterate over the independent components
    for idx in klmn_indpt:
        ref = dijk[idx[0], idx[1], idx[2]]
        # Iterate over the components that should be equivalent to the reference under Kleinman
        for tmp in klmn_equival:
            str = tmp.replace("i", f"{idx[0]}").replace("j", f"{idx[1]}").replace("k", f"{idx[2]}")
            idx_equival = [int(str[0]),int(str[1]),int(str[2])]
            compared = dijk[idx_equival[0], idx_equival[1], idx_equival[2]]

            if not math.isclose(ref, compared, abs_tol=1e-09) or (np.sign(ref*compared)==-1 and ref>=1e-09):
                print(f"The SHG tensor does not respect Kleinman symmetry due to the following components\
                      being different: d{idx}={ref} and d{idx_equival}={compared}")
                return False

    return True

def create_kleinman_tensor(d11=0, d12=0, d13=0, d14=0, d15=0, d16=0, d22=0, d23=0, d24=0, d33=0, as_voigt=True):
    d = np.array([  [d11, d12, d13, d14, d15, d16],
                    [d16, d22, d23, d24, d14, d12],
                    [d15, d24, d33, d23, d13, d14]
                  ])

    if as_voigt:
        return d

    return shg.from_voigt(d)

In [237]:
def pg_nb_to_symbol(pg_symbol=None, pg_nb=None):
    pg_symbols_HM = ["1", "-1", "2", "m", "2/m", "222", "mm2", "mmm", "4", "-4", "4/m", "422", "4mm", "-42m", "4/mmm", "3", 
                     "-3", "32", "3m", "-3m", "6", "-6", "6/m", "622", "6mm", "-6m2", "6/mmm", "23", "m-3", "432", "-43m", 
                     "m-3m"]
    cs_pg_symbols_HM = ["-1", "2/m", "mmm", "4/m", "4/mmm", "-3", "-3m", "6/m", "6/mmm", "m-3", "m-3m"]

    if not pg_nb and not pg_symbol:
        raise Exception("No argument has been provided.")
    if pg_nb and pg_symbol:
        raise Exception("Please provide either the point group number or its symbol, not both.")
    if pg_symbol and pg_symbol not in pg_symbols_HM:
        raise Exception(f"The point group symbol {pg_symbol} provided as argument is not valid.")

    if pg_nb:
        pg = pg_symbols_HM[pg_nb-1]
    else:
        pg = pg_symbol

    if pg in cs_pg_symbols_HM:
        pg = "cs_pg"

    return pg

In [238]:
def get_mask_shg_kleinman_by_pg(pg_nb=None, pg_symbol=None, keep_null=False, as_voigt=True, spg_symbol=None):
    pg = pg_nb_to_symbol(pg_symbol=pg_symbol, pg_nb=pg_nb)

    shg_shape_by_pg = {
        "1":    {"d11": 1, "d12": 1, "d13": 1, "d14": 1, "d15": 1, "d16": 1, "d22": 1, "d23": 1, "d24": 1, "d33": 1},
        "2":    {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 0, "d16": 1, "d22": 1, "d23": 1, "d24": 0, "d33": 0},
        "m":    {"d11": 1, "d12": 1, "d13": 1, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "222":  {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "mm2":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "4":    {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "-4":   {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 0},
        "422":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "4mm":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "-42m": {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "3":    {"d11": 1, "d12": 1, "d13": 0, "d14": 0, "d15": 1, "d16": 1, "d22": 1, "d23": 0, "d24": 1, "d33": 1},
        "32":   {"d11": 1, "d12": 1, "d13": 0, "d14": 0, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "3m":   {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 1, "d22": 1, "d23": 0, "d24": 1, "d33": 1},
        "6":    {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "-6":   {"d11": 1, "d12": 1, "d13": 0, "d14": 0, "d15": 0, "d16": 1, "d22": 1, "d23": 0, "d24": 0, "d33": 0},
        "622":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "6mm":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 1, "d16": 0, "d22": 0, "d23": 0, "d24": 1, "d33": 1},
        "-6m2": {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 0, "d16": 1, "d22": 1, "d23": 0, "d24": 0, "d33": 0},
        "23":   {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "432":  {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "-43m": {"d11": 0, "d12": 0, "d13": 0, "d14": 1, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
        "cs_pg": {"d11": 0, "d12": 0, "d13": 0, "d14": 0, "d15": 0, "d16": 0, "d22": 0, "d23": 0, "d24": 0, "d33": 0},
    }

    mask = create_kleinman_tensor(**shg_shape_by_pg[pg], as_voigt=as_voigt)

    # Handle spg which have axes inverted wrt usual pg (to fill when I encounter entries like this)
    if spg_symbol and pg=='32' and spg_symbol in ["P312", "P3_112", "P3_212"]:
        mask = np.abs(np.round(shg.to_voigt(shg.apply_rot(mask, 0, 0, 90))))
    elif spg_symbol and pg=='-42m' and spg_symbol in ["I-4m2", "P-4m2", "I-4c2",]:
        mask = np.abs(np.round(shg.to_voigt(shg.apply_rot(mask, 0, 0, 45))))
    elif spg_symbol and pg=='3m' and spg_symbol in ["P31m", "P31c"]:
        mask = np.abs(np.round(shg.to_voigt(shg.apply_rot(mask, 0, 0, 90))))
    elif spg_symbol and pg=='-6m2' and spg_symbol in ["P-62m", "P-62c"]:
        mask = np.abs(np.round(shg.to_voigt(shg.apply_rot(mask, 0, 0, 90))))

    # If one wants a mask to keep the components that are supposed to be zero
    if keep_null:
        # mask =  ~create_kleinman_tensor(**shg_shape_by_pg[pg], as_voigt=as_voigt)+2
        mask = ~mask.astype(np.int32) + 2

    return mask

# Return True if the non-null independent components are properly related by neumann principle for each point group
# This requires the SHG tensor to adopt the conventional form.
def verify_shg_neumann_by_pg(d, pg_nb=None, pg_symbol=None, abs_tol=1e-1, rel_tol_coef=0.05, spg_symbol=None):
    pg = pg_nb_to_symbol(pg_symbol=pg_symbol, pg_nb=pg_nb)

    dijk = shg.to_voigt(d)

    # Handle spg which have axes inverted wrt usual pg (to fill when I encounter entries like this)
    if spg_symbol in ["P312", "P3_112", "P3_212"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))
    elif spg_symbol in ["I-4m2", "P-4m2", "I-4c2",]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 45))
    elif spg_symbol in ["P31m", "P31c"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))
    elif spg_symbol in ["P-62m", "P-62c"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))

    d11 =  dijk[1-1,1-1] ; d12 =  dijk[1-1,2-1] ; d13 =  dijk[1-1,3-1] ; d14 =  dijk[1-1,4-1] ; d15 =  dijk[1-1,5-1]
    d16 =  dijk[1-1,6-1] ; d22 =  dijk[2-1,2-1] ; d23 =  dijk[2-1,3-1] ; d24 =  dijk[2-1,4-1] ; d33 =  dijk[3-1,3-1]

    if pg in ["1", "2", "m", "222", "mm2", "422", "-42m", "622", "23", "432", "-43m", "cs_pg"]:
        # No constraints are imposed on the independent components for these pg
        tmp = [True]
    elif pg in ["4", "4mm", "6", "6mm"]:
        if rel_tol_coef:
            rel_tol = rel_tol_coef*np.max([np.abs(d15), np.abs(d24)])
        else:
            rel_tol = 1e-9 # default value of math.isclose
        tmp = [math.isclose(np.abs(d15), np.abs(d24), abs_tol=abs_tol, rel_tol=rel_tol)]
    elif pg=="-4":
        if rel_tol_coef:
            rel_tol = rel_tol_coef*np.max([np.abs(d15), np.abs(d24)])
        else:
            rel_tol = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d15, -d24, abs_tol=abs_tol, rel_tol=rel_tol),]
    elif pg in ["3"]:
        if rel_tol_coef:
            rel_tol_1 = rel_tol_coef*np.max([np.abs(d16), np.abs(d22)])
            rel_tol_2 = rel_tol_coef*np.max([np.abs(d11), np.abs(d12)])
            rel_tol_3 = rel_tol_coef*np.max([np.abs(d15), np.abs(d24)])
        else:
            rel_tol_1 = 1e-9 # default value of math.isclose
            rel_tol_2 = 1e-9 # default value of math.isclose
            rel_tol_3 = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d16, -d22, abs_tol=abs_tol, rel_tol=rel_tol_1),
               math.isclose(d11, -d12, abs_tol=abs_tol, rel_tol=rel_tol_2),
               math.isclose(np.abs(d15), np.abs(d24), abs_tol=abs_tol, rel_tol=rel_tol_3)]
    elif pg in ["-6"]:
        if rel_tol_coef:
            rel_tol_1 = rel_tol_coef*np.max([np.abs(d16), np.abs(d22)])
            rel_tol_2 = rel_tol_coef*np.max([np.abs(d11), np.abs(d12)])
        else:
            rel_tol_1 = 1e-9 # default value of math.isclose
            rel_tol_2 = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d16, -d22, abs_tol=abs_tol, rel_tol=rel_tol_1),
               math.isclose(d11, -d12, abs_tol=abs_tol, rel_tol=rel_tol_2)]
    elif pg=="32":
        if rel_tol_coef:
            rel_tol = rel_tol_coef*np.max([np.abs(d11), np.abs(d12)])
        else:
            rel_tol = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d11, -d12, abs_tol=abs_tol, rel_tol=rel_tol),]
    elif pg=="3m":
        if rel_tol_coef:
            rel_tol_1 = rel_tol_coef*np.max([np.abs(d16), np.abs(d22)])
            rel_tol_2 = rel_tol_coef*np.max([np.abs(d15), np.abs(d24)])
        else:
            rel_tol_1 = 1e-9 # default value of math.isclose
            rel_tol_2 = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d16, -d22, abs_tol=abs_tol, rel_tol=rel_tol_1),
               math.isclose(np.abs(d15), np.abs(d24), abs_tol=abs_tol, rel_tol=rel_tol_2)]
    elif pg=="-6m2":
        if rel_tol_coef:
            rel_tol = rel_tol_coef*np.max([np.abs(d16), np.abs(d22)])
        else:
            rel_tol = 1e-9 # default value of math.isclose
        tmp = [math.isclose(d16, -d22, abs_tol=abs_tol, rel_tol=rel_tol),]
    else:
        raise Exception(f"The point group symbol {pg} is unknown...")

    return all(tmp)


In [239]:
def equate_coef(d1, d2):
    d_com = np.mean([np.abs(d1), np.abs(d2)])
    return np.sign(d1)*d_com, np.sign(d2)*d_com

def correct_shg_neumann_by_pg(d, pg_nb=None, pg_symbol=None, spg_symbol=None):
    pg = pg_nb_to_symbol(pg_symbol=pg_symbol, pg_nb=pg_nb)

    dijk = shg.to_voigt(d)

    # Handle spg which have axes inverted wrt usual pg (to fill when I encounter entries like this)
    if spg_symbol in ["P312", "P3_112", "P3_212"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))
    elif spg_symbol in ["I-4m2", "P-4m2", "I-4c2",]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 45))
    elif spg_symbol in ["P31m", "P31c"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))
    elif spg_symbol in ["P-62m", "P-62c"]:
        dijk = shg.to_voigt(shg.apply_rot(dijk, 0, 0, 90))

    d11 =  dijk[1-1,1-1] ; d12 =  dijk[1-1,2-1] ; d13 =  dijk[1-1,3-1] ; d14 =  dijk[1-1,4-1] ; d15 =  dijk[1-1,5-1]
    d16 =  dijk[1-1,6-1] ; d22 =  dijk[2-1,2-1] ; d23 =  dijk[2-1,3-1] ; d24 =  dijk[2-1,4-1] ; d33 =  dijk[3-1,3-1]

    if pg in ["1", "2", "m", "222", "mm2", "422", "-42m", "622", "23", "432", "-43m", "cs_pg"]:
        # No constraints are imposed on the independent components for these pg
        tmp = [True]
    elif pg in ["4", "4mm", "6", "6mm"]:
        d15, d24 = equate_coef(d15, d24)
    elif pg=="-4":
        d15, d24 = equate_coef(d15, d24)
    elif pg in ["3"]:
        d16, d22 = equate_coef(d16, d22)
        d11, d12 = equate_coef(d11, d12)
        d15, d24 = equate_coef(d15, d24)
    elif pg in ["-6"]:
        d16, d22 = equate_coef(d16, d22)
        d11, d12 = equate_coef(d11, d12)
    elif pg=="32":
        d11, d12 = equate_coef(d11, d12)
    elif pg=="3m":
        d16, d22 = equate_coef(d16, d22)
        d15, d24 = equate_coef(d15, d24)
    elif pg=="-6m2":
        d16, d22 = equate_coef(d16, d22)
    else:
        raise Exception(f"The point group symbol {pg} is unknown...")

    d_constrained = np.array([
        [d11, d12, d13, d14, d15, d16],
        [d16, d22, d23, d24, d14, d12],
        [d15, d24, d33, d23, d13, d14],
    ])

    # Handle spg which have axes inverted wrt usual pg (to fill when I encounter entries like this)
    if spg_symbol in ["P312", "P3_112", "P3_212"]:
        d_constrained = shg.to_voigt(shg.apply_rot(d_constrained, 0, 0, -90))
    elif spg_symbol in ["I-4m2", "P-4m2", "I-4c2",]:
        d_constrained = shg.to_voigt(shg.apply_rot(d_constrained, 0, 0, -45))
    elif spg_symbol in ["P31m", "P31c"]:
        d_constrained = shg.to_voigt(shg.apply_rot(d_constrained, 0, 0, -90))
    elif spg_symbol in ["P-62m", "P-62c"]:
        d_constrained = shg.to_voigt(shg.apply_rot(d_constrained, 0, 0, -90))

    return d_constrained


In [240]:
def get_max_null_component(d, pg_symbol=None, pg_nb=None, spg_symbol=None):
    mask_null = get_mask_shg_kleinman_by_pg(pg_nb=pg_nb, pg_symbol=pg_symbol, keep_null=True, spg_symbol=spg_symbol)
    d_null = shg.to_voigt(d) * mask_null
    return np.max(np.abs(d_null))

def get_sum_full_component(d, pg_symbol=None, pg_nb=None, spg_symbol=None):
    mask_full = get_mask_shg_kleinman_by_pg(pg_nb=pg_nb, pg_symbol=pg_symbol, keep_null=False, spg_symbol=spg_symbol)
    d_full = shg.to_voigt(d) * mask_full
    return np.sum(np.abs(d_full))

def get_full_d(d, pg_symbol=None, pg_nb=None, spg_symbol=None):
    mask_full = get_mask_shg_kleinman_by_pg(pg_nb=pg_nb, pg_symbol=pg_symbol, keep_null=False, spg_symbol=spg_symbol)
    return shg.to_voigt(d) * mask_full

def get_min_full_component(d, pg_symbol=None, pg_nb=None, spg_symbol=None):
    mask_full = get_mask_shg_kleinman_by_pg(pg_nb=pg_nb, pg_symbol=pg_symbol, keep_null=False, spg_symbol=spg_symbol)
    d_full = shg.to_voigt(d) * mask_full
    min_d_full = np.ma.masked_equal(np.abs(d_full), 0.0, copy=False).min()
    if isinstance(min_d_full, np.ma.core.MaskedConstant):
        return 0.0
    return min_d_full

def get_max_full_component(d, pg_symbol=None, pg_nb=None, spg_symbol=None):
    mask_full = get_mask_shg_kleinman_by_pg(pg_nb=pg_nb, pg_symbol=pg_symbol, keep_null=False, spg_symbol=spg_symbol)
    d_full = shg.to_voigt(d) * mask_full
    idx_arg_max = np.unravel_index(np.argmax(np.abs(d_full)), d_full.shape)
    max_d_full = d_full[idx_arg_max]
    return idx_arg_max, max_d_full

# https://stackoverflow.com/questions/58065055/floor-and-ceil-with-number-of-decimals
def my_floor(a, precision=2):
    return np.true_divide(np.floor(a * 10**precision), 10**precision)

def verify_shg_conventional_shape_by_pg(d, pg_nb=None, pg_symbol=None, tol_max_null = 1e-09, tol_rel_null_full=1e-04, verify_neumann=True, spg_symbol=None):
    is_kleinman = verify_kleinman(d)
    if not is_kleinman:
        # print("bc of kleinman")
        return is_kleinman, "Kleinman"

    min_d_full = get_min_full_component(d=d, pg_symbol=pg_symbol, pg_nb=pg_nb, spg_symbol=spg_symbol)
    if math.isclose(min_d_full,0):
        # print(f"bc of min_d_full: {min_d_full}")
        return False, f"{min_d_full = }"

    max_d_null = get_max_null_component(d, pg_symbol=pg_symbol, pg_nb=pg_nb, spg_symbol=spg_symbol)
    
    if max_d_null>=tol_max_null and my_floor(max_d_null/min_d_full, precision=str(tol_rel_null_full)[::-1].find('.'))>tol_rel_null_full:
        # print(f"The largest of the components supposed to be null is {max_d_null} instead of being close to 0 and is greater than 1% of the minimum value of the non-null components.")
        return False, f"{max_d_null/min_d_full = }"

    if verify_neumann:
        is_neumann = verify_shg_neumann_by_pg(d, pg_nb=pg_nb, pg_symbol=pg_symbol, spg_symbol=spg_symbol)
        if not is_neumann:
            # print(f"The given SHG tensor does not respect neumann symmetry.")
            return is_neumann, "neumann"

    return True, None


In [241]:
def apply_all_rot_get_max_null_and_min_full( d, 
                                step      = 10, 
                                minangle_a  = 0,
                                maxangle_a  = 120,
                                minangle_b  = 0,
                                maxangle_b  = 120,
                                minangle_g  = 0,
                                maxangle_g  = 120,
                                pg_symbol = None,
                                pg_nb     = None
                        ):

    # Create the arrays of rotational angles around each axis, from 0 to maxangle included with a step of step
    valpha    = np.arange(minangle_a,maxangle_a+0.5,step)
    vbeta     = np.arange(minangle_b,maxangle_b+0.5,step)
    vgamma    = np.arange(minangle_g,maxangle_g+0.5,step)
    l = len(valpha)

    # Instantiate two arrays with the dKP and dijk for each rotation respectively
    max_null_ar  = np.zeros([l,l,l])
    min_full_ar  = np.zeros([l,l,l])
    
    for ia, alpha in enumerate(valpha):
        for ib, beta in enumerate(vbeta):
            for ig, gamma in enumerate(vgamma):
                d_rot = shg.apply_rot(d, alpha, beta, gamma)
                max_null_ar[ia, ib, ig] = get_max_null_component(d_rot, pg_symbol=pg_symbol, pg_nb=pg_nb)
                min_full_ar[ia, ib, ig] = get_min_full_component(d_rot, pg_symbol=pg_symbol, pg_nb=pg_nb)
    
    
    return valpha,vbeta,vgamma,max_null_ar, min_full_ar

def apply_all_rot_get_max_null( d, 
                                step      = 10, 
                                minangle_a  = 0,
                                maxangle_a  = 120,
                                minangle_b  = 0,
                                maxangle_b  = 120,
                                minangle_g  = 0,
                                maxangle_g  = 120,
                                pg_symbol = None,
                                pg_nb     = None
                        ):

    # Create the arrays of rotational angles around each axis, from 0 to maxangle included with a step of step
    valpha    = np.arange(minangle_a,maxangle_a+0.5,step)
    vbeta     = np.arange(minangle_b,maxangle_b+0.5,step)
    vgamma    = np.arange(minangle_g,maxangle_g+0.5,step)
    l = len(valpha)

    # Instantiate two arrays with the dKP and dijk for each rotation respectively
    max_null_ar  = np.zeros([l,l,l])
    
    for ia, alpha in enumerate(valpha):
        for ib, beta in enumerate(vbeta):
            for ig, gamma in enumerate(vgamma):
                d_rot = shg.apply_rot(d, alpha, beta, gamma)
                max_null_ar[ia, ib, ig] = get_max_null_component(d_rot, pg_symbol=pg_symbol, pg_nb=pg_nb)
    
    
    return valpha,vbeta,vgamma,max_null_ar


def minimize_max_null(  d,
                        pg_symbol = None,
                        pg_nb     = None,
                        step=5,
                        minangle_a  = 0,
                        maxangle_a  = 120,
                        minangle_b  = 0,
                        maxangle_b  = 120,
                        minangle_g  = 0,
                        maxangle_g  = 120,
                        ):

    valpha,vbeta,vgamma,max_null_ar = apply_all_rot_get_max_null(   d,
                                                                    step=step,
                                                                    minangle_a  = minangle_a,
                                                                    maxangle_a  = maxangle_a,
                                                                    minangle_b  = minangle_b,
                                                                    maxangle_b  = maxangle_b,
                                                                    minangle_g  = minangle_g,
                                                                    maxangle_g  = maxangle_g,
                                                                    pg_symbol=pg_symbol,
                                                                    pg_nb=pg_nb)
    idx_arg_min = np.unravel_index(np.argmin(max_null_ar), max_null_ar.shape)
    min_a = valpha[idx_arg_min[0]]
    min_b = vbeta[idx_arg_min[1]]
    min_g = vgamma[idx_arg_min[2]]
    min_max_null = max_null_ar.min()

    return min_a, min_b, min_g, min_max_null

In [242]:
def switch_sign_max_full(d, pg_symbol=None, pg_nb=None):
    pg = pg_nb_to_symbol(pg_symbol=pg_symbol, pg_nb=pg_nb)
    if pg == "cs_pg":
        raise Exception(f"The point group {pg} is centrosymmetric... d_ijk should all be zero.")
    idx_arg_max, max_d_full = get_max_full_component(d, pg_symbol=pg)
    max_d_null = get_max_null_component(d, pg_symbol=pg)

    if np.sign(max_d_full)==1:
        return (0.0, 0.0, 0.0, max_d_full)

    for a, b, g in [(180.0, 0.0, 0.0),      (0.0, 180.0, 0.0),      (0.0, 0.0, 180.0),
                    (180.0, 180.0, 0.0),    (180.0, 0.0, 180.0),    (0.0, 180.0, 180.0),
                    (90.0, 0.0, 0.0),      (0.0, 90.0, 0.0),      (0.0, 0.0, 90.0),
                    (90.0, 90.0, 0.0),    (90.0, 0.0, 90.0),    (0.0, 90.0, 90.0),
                    ]:
        d_rot = shg.apply_rot(d, a, b, g)
        max_d_full_rot = shg.to_voigt(d_rot)[idx_arg_max]
        if np.sign(max_d_full_rot)==1:
            if not math.isclose(max_d_full_rot, np.abs(max_d_full), abs_tol=1e-06):
                raise Exception(f"Once rotated to switch the sign of the max. of the abs. values of dijk to +, the latter is different than originally: {max_d_full_rot}!={max_d_full}")
            max_d_rot_null = get_max_null_component(d_rot, pg_symbol=pg)
            if math.isclose(max_d_null, max_d_rot_null, abs_tol=1e-3):
                return (a, b, g, max_d_full_rot)

    print(pg_symbol)
    print(d)
    print(max_d_full)
    raise Exception(f"The sign of the maximum of the absolute values of the dijk could not be switched to positive...")

In [243]:
def apply_rot_by_R(d, R):

    # Define the order of the tensor to rotate
    ndim = len(d.shape)
        
    # Equivalent to R @ d @ R.T but generalized to any order of tensor
    # Rotate d: use np.einsum to multiply d by R in a for-loop over the order of tensor to target each dimension
    subs = 'ijklmnop'[:ndim]
    eins = 'Zz,{}->{}'
    for n in range(ndim): # or reversed(range(ndim)), does not matter
        eins_n = eins.format(subs.replace(subs[n],'z'),subs.replace(subs[n],'Z')) # 'Zz,ijz->ijZ' then 'Zz,izk->iZk' then 'Zz,zjk->Zjk' for ndim=3
        d = np.einsum(eins_n,R,d)
    
    return d

# Applies a Euler rotation to a system of vectors such as rprim e.g. (each row of syst is a vector to be rotated) ===============
# Rotation of alpha around x, then beta around new y, then gamma around new z (angles in degrees!)
# Throws an Exception/Error if the arguments are not of the same type as the annotations
# syst can have any number of row but each row must be 3 elements long
def apply_rot_syst_by_R(syst,
                        R
                        ):

    # Decomposes the rotation by rotating each vector (1st order tensor) individually 
    syst = shg.to_array(syst)
    syst_rot = np.zeros(np.shape(syst))
    for i in range(len(syst)):
        syst_rot[i] = apply_rot_by_R(syst[i],R)

    return syst_rot

In [244]:
# Plot an ellipsoid with semi-axis length a along x, b (y), and c (z) and then rotate this ellispoid by (alpha, beta, gamma)
def plot_ellipsoid(diag_indicatrix, t=None, alpha=0, beta=0, gamma=0, add_alpha=0, add_beta=0, add_gamma=0):

    # some math: generate points on the surface of the ellipsoid
    
    # Indicatrix
    a = diag_indicatrix[0]
    b = diag_indicatrix[1]
    c = diag_indicatrix[2]

    phi = np.linspace(0, 2*np.pi)
    theta = np.linspace(-np.pi/2, np.pi/2)
    phi, theta=np.meshgrid(phi, theta)
    
    x = np.cos(theta) * np.sin(phi) * a
    y = np.cos(theta) * np.cos(phi) * b
    z = np.sin(theta) * c

    lim_range = np.max([a,b,c])*1.25

    points_ellips = np.array([x.flatten(), y.flatten(), z.flatten()]).T # row = 1 point, column=x,y,z // rotation lattice structure
    points_ellips_rotated_bis = shg.apply_rot_syst(points_ellips, alpha, beta, gamma)
    points_ellips_rotated = shg.apply_rot_syst(points_ellips_rotated_bis, add_alpha, add_beta, add_gamma)
    
    fig = go.Figure(data=[Mesh3d({
                                   'x': points_ellips_rotated[:,0],
                                   'y': points_ellips_rotated[:,1],
                                   'z': points_ellips_rotated[:,2],
                                   'alphahull': 0
                                   })
                        ]
                    )                  
    

    fig.add_trace(go.Scatter3d(x=[-lim_range, lim_range], 
                               y=[0, 0],
                               z=[0, 0],
                               mode='lines', line_width = 15, line_color = "black"))
    fig.add_trace(go.Scatter3d(x=[0, 0], 
                               y=[-lim_range, lim_range],
                               z=[0, 0],
                               mode='lines', line_width = 15, line_color = "black"))
    fig.add_trace(go.Scatter3d(x=[0, 0], 
                               y=[0, 0],
                               z=[-lim_range, lim_range],
                               mode='lines', line_width = 15, line_color = "black"))

    # Optical axis
    if t!=None:
        coord_line_z = 3.0*np.max(z)*np.cos(np.deg2rad(t)) # *1.5 to make it visible despite the ellipsoid
        coord_line_x = 3.0*np.max(z)*np.sin(np.deg2rad(t))
        coords_line = np.array(
            [
                [-coord_line_x,     0,      -coord_line_z],
                [coord_line_x,      0,       coord_line_z]
            ]
        )
        coords_line_rotated_bis = shg.apply_rot_syst(coords_line, alpha, beta, gamma)
        coords_line_rotated = shg.apply_rot_syst(coords_line_rotated_bis, add_alpha, add_beta, add_gamma)
        print(f"{coords_line_rotated = }")

        fig.add_trace(go.Scatter3d(x=coords_line_rotated[:,0], 
                                   y=coords_line_rotated[:,1],
                                   z=coords_line_rotated[:,2],
                                   mode='lines', line_width = 15, line_color = "red"))
        coords_line = np.array(
            [
                [coord_line_x,      0,      -coord_line_z],
                [-coord_line_x,     0,      coord_line_z]
            ]
        )
        coords_line_rotated_bis = shg.apply_rot_syst(coords_line, alpha, beta, gamma)
        coords_line_rotated = shg.apply_rot_syst(coords_line_rotated_bis, add_alpha, add_beta, add_gamma)
        print(f"{coords_line_rotated = }")

        fig.add_trace(go.Scatter3d(x=coords_line_rotated[:,0], 
                                   y=coords_line_rotated[:,1],
                                   z=coords_line_rotated[:,2],
                                   mode='lines', line_width = 15, line_color = "red"))

    # Layout
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[-lim_range, lim_range]), # nticks=4
            yaxis=dict(range=[-lim_range, lim_range]),
            zaxis=dict(range=[-lim_range, lim_range])
        ),
    )
    fig.update_layout(scene_aspectmode='cube')
    fig.update_layout(scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="rgb(230, 230,200)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",),),
                    width=700,
                    margin=dict(
                    r=10, l=10,
                    b=10, t=10)
                  )
    
    
    init_notebook_mode()
    iplot(fig)

# Load latest SHG dataset

In [245]:
df = pd.read_pickle("/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/shg_nov24/01_humanguided_al_v1/20_al_it/data/df_outputs_filtout.pkl.gz")
print(df.shape)
display(df.head())

(2699, 26)


,formula_reduced,crystal_system,dRMS,dijk,elements,epsij,src_bandgap,src_DB_IDs,src_ehull,src_epsij,...,nsites,pg_symbol,spg_number,spg_symbol,structure,dKP,src,origin,dinv2,dinv3
mp-34078,H3ClO,trigonal,0.554479,"[[[0.0, 1.232027588815095, -0.374056043202926]...","[Cl, H, O]","[[2.53058371, -0.0, -0.0], [-0.0, 2.53058371, ...",5.3899,{},0.005834,"[[2.486930581530363, -5.922864326768362e-07, -...",...,5,3m,160,R3m,"{'@module': 'pymatgen.core.structure', '@class...",0.999857,Materials Project,Naccarato,1.707401,2.065073
mp-632326,HCl,orthorhombic,0.653691,"[[[0.0, 0.0, 0.0027236759621490003], [0.0, 0.0...","[Cl, H]","[[2.15315296, -0.0, -0.0], [-0.0, 2.33339089, ...",5.9709,{'icsd': ['icsd-27037']},0.000000,"[[2.05131012, 0.0, 0.0], [0.0, 1.80706055, 0.0...",...,4,mm2,36,Cmc2_1,"{'@module': 'pymatgen.core.structure', '@class...",1.516039,Materials Project,Naccarato,2.075681,2.917119
mp-3277,BAsO4,tetragonal,1.631147,"[[[0.032043250933791005, 0.032043250941036, 1....","[As, B, O]","[[2.93707095, -0.0, -0.0], [-0.0, 2.93707095, ...",4.4406,"{'icsd': ['icsd-413438', 'icsd-26891']}",0.000000,"[[3.0300598691968603, -1.4644051369749889e-05,...",...,6,-4,82,I-4,"{'@module': 'pymatgen.core.structure', '@class...",2.924868,Materials Project,Naccarato,3.462290,5.880784
mp-570935,LiI,hexagonal,0.103603,"[[[0.0, 0.0, -0.061151779065493], [0.0, 0.0, 2...","[I, Li]","[[3.26178546, -0.0, -0.0], [-0.0, 3.26178546, ...",4.3810,{'icsd': ['icsd-414242']},0.000000,"[[3.1340718616738847, 3.9628310656241467e-07, ...",...,4,6mm,186,P6_3mc,"{'@module': 'pymatgen.core.structure', '@class...",0.244556,Materials Project,Naccarato,0.650335,0.524261
mp-20459,TiPbO3,tetragonal,10.949745,"[[[0.0, 0.0, -23.192675819051072], [0.0, 0.0, ...","[O, Pb, Ti]","[[7.09469421, -0.0, -0.0], [-0.0, 7.09469421, ...",1.9569,"{'icsd': ['icsd-1612', 'icsd-61168', 'icsd-550...",0.000000,"[[6.598517479999999, 0.0, 0.0], [0.0, 6.598517...",...,5,4mm,99,P4mm,"{'@module': 'pymatgen.core.structure', '@class...",23.177082,Materials Project,Naccarato,12.317948,39.450222


# Compare spg with initial spg from agm and mp

I noticed that certain materials have a mismatch between their spacegroup retrieved after calculation and their spacegroup from Alexandria or the MP. This mismatch has 2 causes:

1. Abinit (and abipy) use a stricter tolerance on the symmetry (symprec=0.001 for abipy) when abi_sanitizing the structure and detecting the spg than pymatgen (default symprec=0.01). So it can happen that the calculated space group is lower than the announced one (or detected afterwards with pymatgen).
2. The Alexandria materials of the first AL iterations and VT-2k come from the PBEsol subset while the other iterations come from the PBE subset. Unfortunately, while sharing the same agm-id, a material can have 2 different spg due to the difference in relaxation... (e.g., agm003172890 or agm003128264)

To alleviate this issue, I have decided to define the space group by strictly using the space group detected by Abinit:
```
structure = Structure.from_dict(r['structure'])
dict_spginfo = structure.abiget_spginfo()
spg_symbol = dict_spginfo['spg_symbol'].split("(#")[0]
spg_number = dict_spginfo['spg_number']
```
So below, I need to determine the ABINIT spg and pg for all my compounds because it highly matters when searching for the conventional form of the SHG tensor.

In [246]:
# path_df_std = '/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/shg_nov24/00_prepa_dbs/agm/data/df_agm_nov24_std.pkl.gz'
# df_agm = pd.read_pickle(path_df_std)
# print(f"{df_agm.shape = }")
# print(f"{df_agm.columns = }")

# path_df_std = '/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/shg_nov24/00_prepa_dbs/mp/data/df_mp_nov24_std.pkl.gz'
# df_mp = pd.read_pickle(path_df_std)
# print(f"{df_mp.shape = }")
# print(f"{df_mp.columns = }")

# list_spg_notmatch = []
# for ir, r in df.iterrows():
#     if "agm" in ir:
#         df_tmp = df_agm
#     elif "mp" in ir:
#         df_tmp = df_mp

#     try:
#         spg_src = df_tmp.loc[ir]['spg']
#     except KeyError:
#         continue
#     spg_now = r['spg_number']
#     if spg_now != spg_src:
#         list_spg_notmatch.append(ir)

# df_spg_notmatch = df.filter(list_spg_notmatch, axis=0)
# print(len(df_spg_notmatch))
# print(df_spg_notmatch['src'].value_counts())
# 145
# src
# Alexandria           132
# Materials Project     10
# VT-2k                  3

# Re-determine the spg and pg

In [247]:
path_df = "/home/vtrinquet/Documents/Doctorat/JNB_Scripts_Clusters/NLO/HT/shg_nov24/01_humanguided_al_v1/20_al_it/data/df_outputs_filtout_abipg.pkl.gz"
if Path(path_df).exists():
    df = pd.read_pickle(path_df)
else:
    # to use PointGroup.from_space_group I had to update my pymatgen version... before I was using the following:
    # $ pip freeze | grep pymatgen
    # pymatgen==2024.3.1
    from pymatgen.symmetry.groups import PointGroup
    from abipy.core.structure import Structure as AbiStructure
    list_abi_spg_symbol = []
    list_abi_spg_number = []
    list_abi_pg_symbol = []
    list_abi_crystal_system = []
    for ir, r in df.iterrows():
        structure = AbiStructure.from_dict(r['structure'])
        dict_spginfo = structure.abiget_spginfo()
        list_abi_spg_symbol.append(dict_spginfo['spg_symbol'].split("(#")[0])
        list_abi_spg_number.append(dict_spginfo['spg_number'])
        list_abi_crystal_system.append(dict_spginfo['bravais'].split("(")[1][:-1])
        try:
            list_abi_pg_symbol.append(PointGroup.from_space_group(sg_symbol=list_abi_spg_symbol[-1]).symbol)
        except ValueError:
            # Some matching between spg and pg are unknown to pymatgen, so need to do this matching manually via 
            # Wikipedia list of spg
            if list_abi_spg_symbol[-1] in ["P3_221", "P3_121", "P321"]:
                list_abi_pg_symbol.append(PointGroup("32").symbol)
            elif list_abi_spg_symbol[-1] in ["P3m1"]:
                list_abi_pg_symbol.append(PointGroup("3m").symbol)
            else:
                print(list_abi_spg_symbol[-1])

    df['abi_spg_symbol'] = list_abi_spg_symbol
    df['abi_spg_number'] = list_abi_spg_number
    df['abi_pg_symbol'] = list_abi_pg_symbol
    df['abi_crystal_system'] = list_abi_crystal_system

    df.to_pickle(path_df)

print(df.shape)
print(df.columns)

(2699, 30)
Index(['formula_reduced', 'crystal_system', 'dRMS', 'dijk', 'elements',
       'epsij', 'src_bandgap', 'src_DB_IDs', 'src_ehull', 'src_epsij',
       'src_is_gap_direct', 'src_is_magnetic', 'src_n', 'src_theoretical', 'n',
       'nelements', 'nsites', 'pg_symbol', 'spg_number', 'spg_symbol',
       'structure', 'dKP', 'src', 'origin', 'dinv2', 'dinv3', 'abi_spg_symbol',
       'abi_spg_number', 'abi_pg_symbol', 'abi_crystal_system'],
      dtype='object')


# Replace the structure of the Naccarato entries by the original structures of his dataset

In [248]:
path_df = "data/df_nac_origin.pkl.gz"
if Path(path_df).exists():
    df_nac_origin = pd.read_pickle(path_df)
else:
    df_nac_origin = pd.read_json("data/DB_NLO_FN.json")
    df_nac_origin.index = df_nac_origin['MP_id']
    df_nac_origin = df_nac_origin.drop(['MP_id'], axis=1)
    list_structure = []
    for ir, r in df_nac_origin.iterrows():
        list_structure.append(Structure.from_str(input_string=r['input_params']['structure'], fmt="cif").as_dict())
    df_nac_origin['structure'] = list_structure

    from pymatgen.symmetry.groups import PointGroup
    from abipy.core.structure import Structure as AbiStructure
    list_abi_spg_symbol = []
    list_abi_spg_number = []
    list_abi_pg_symbol = []
    list_abi_crystal_system = []
    for ir, r in df_nac_origin.iterrows():
        structure = AbiStructure.from_dict(r['structure'])
        dict_spginfo = structure.abiget_spginfo()
        list_abi_spg_symbol.append(dict_spginfo['spg_symbol'].split("(#")[0])
        list_abi_spg_number.append(dict_spginfo['spg_number'])
        list_abi_crystal_system.append(dict_spginfo['bravais'].split("(")[1][:-1])
        try:
            list_abi_pg_symbol.append(PointGroup.from_space_group(sg_symbol=list_abi_spg_symbol[-1]).symbol)
        except ValueError:
            # Some matching between spg and pg are unknown to pymatgen, so need to do this matching manually via 
            # Wikipedia list of spg
            if list_abi_spg_symbol[-1] in ["P3_221", "P3_121", "P321"]:
                list_abi_pg_symbol.append(PointGroup("32").symbol)
            elif list_abi_spg_symbol[-1] in ["P3m1"]:
                list_abi_pg_symbol.append(PointGroup("3m").symbol)
            else:
                print(list_abi_spg_symbol[-1])

    df_nac_origin['abi_spg_symbol'] = list_abi_spg_symbol
    df_nac_origin['abi_spg_number'] = list_abi_spg_number
    df_nac_origin['abi_pg_symbol'] = list_abi_pg_symbol
    df_nac_origin['abi_crystal_system'] = list_abi_crystal_system

    df_nac_origin.to_pickle(path_df)

print(df_nac_origin.shape)
df_nac_origin = df_nac_origin.filter(df[df['origin']=="Naccarato"].index, axis=0)
print(df_nac_origin.shape)
display(df_nac_origin.head())

(579, 11)
(558, 11)


,crystal_system,point_group,space_group,space_group_number,input_params,dte_output,structure,abi_spg_symbol,abi_spg_number,abi_pg_symbol,abi_crystal_system
MP_id,,,,,,,,,,,
mp-34078,trigonal,3m,R3m,160,{'structure': '# generated using pymatgen data...,"{'eps_inf': [[2.53058371, 0, 0], [0, 2.5305837...","{'@module': 'pymatgen.core.structure', '@class...",R3m,160,3m,rhombohedral
mp-632326,orthorhombic,mm2,Cmc2_1,36,{'structure': '# generated using pymatgen data...,"{'eps_inf': [[2.15315296, 0, 0], [0, 2.3333908...","{'@module': 'pymatgen.core.structure', '@class...",Cmc2_1,36,mm2,1-face-center ortho.
mp-3277,tetragonal,-4,I-4,82,{'structure': '# generated using pymatgen data...,"{'eps_inf': [[2.93707095, 0, 0], [0, 2.9370709...","{'@module': 'pymatgen.core.structure', '@class...",I-4,82,-4,body-center tetrag.
mp-570935,hexagonal,6mm,P6_3mc,186,{'structure': '# generated using pymatgen data...,"{'eps_inf': [[3.26178546, 0, 0], [0, 3.2617854...","{'@module': 'pymatgen.core.structure', '@class...",P6_3mc,186,6mm,primitive hexag.
mp-20459,tetragonal,4mm,P4mm,99,{'structure': '# generated using pymatgen data...,"{'eps_inf': [[7.09469421, 0, 0], [0, 7.0946942...","{'@module': 'pymatgen.core.structure', '@class...",P4mm,99,4mm,primitive tetrag.


In [249]:
df.columns

Index(['formula_reduced', 'crystal_system', 'dRMS', 'dijk', 'elements',
       'epsij', 'src_bandgap', 'src_DB_IDs', 'src_ehull', 'src_epsij',
       'src_is_gap_direct', 'src_is_magnetic', 'src_n', 'src_theoretical', 'n',
       'nelements', 'nsites', 'pg_symbol', 'spg_number', 'spg_symbol',
       'structure', 'dKP', 'src', 'origin', 'dinv2', 'dinv3', 'abi_spg_symbol',
       'abi_spg_number', 'abi_pg_symbol', 'abi_crystal_system'],
      dtype='object')

In [250]:
path_df = "data/df_outputs_filtout_nac-replaced.pkl.gz"
if Path(path_df).exists():
    df = pd.read_pickle(path_df)
else:
    for ir, r in df.iterrows():
        if r['origin']=="Naccarato":
            r_nac = df_nac_origin.loc[ir]
            eps = r_nac['dte_output']['eps_inf']
            dte = r_nac['dte_output']['dte']
            structure = Structure.from_dict(r_nac['structure'])
            spga = SpacegroupAnalyzer(structure)

            df.at[ir, 'formula_reduced'] = structure.composition.reduced_formula
            df.at[ir, 'crystal_system'] = spga.get_crystal_system()
            df.at[ir, 'dRMS'] = shg.get_dRMS(dte)
            df.at[ir, 'dRMS'] = shg.get_dKP(dte)
            df.at[ir, 'dijk'] = dte
            df.at[ir, 'epsij'] = eps
            df.at[ir, 'n'] = np.mean(np.linalg.eig(eps)[0])
            df.at[ir, 'elements'] = [str(el) for el in structure.composition.elements]
            df.at[ir, 'nelements'] = len([str(el) for el in structure.composition.elements])
            df.at[ir, 'nsites'] = len(structure)
            df.at[ir, 'pg_symbol'] = spga.get_point_group_symbol()
            df.at[ir, 'spg_number'] = spga.get_space_group_number()
            df.at[ir, 'spg_symbol'] = spga.get_space_group_symbol()
            df.at[ir, 'structure'] = structure.as_dict()
            df.at[ir, 'abi_spg_symbol'] = r_nac['abi_spg_symbol']
            df.at[ir, 'abi_spg_number'] = r_nac['abi_spg_number']
            df.at[ir, 'abi_pg_symbol'] = r_nac['abi_pg_symbol']
            df.at[ir, 'abi_crystal_system'] = r_nac['abi_crystal_system']
    df.to_pickle(path_df)

# Correct the tensor

## Rotation IEEE pmg

In [272]:
class SHGTensor(Tensor):
    @staticmethod
    def get_ieee_rotation(
        structure: Structure,
        refine_rotation: bool = True,
    ) -> SquareTensor:
        """Given a structure associated with a tensor, determines
        the rotation matrix for IEEE conversion according to
        the 1987 IEEE standards.

        Args:
            structure (Structure): a structure associated with the
                tensor to be converted to the IEEE standard
            refine_rotation (bool): whether to refine the rotation
                using SquareTensor.refine_rotation
        """
        # Check conventional setting:
        sga = SpacegroupAnalyzer(structure, symprec=0.001)
        dataset = sga.get_symmetry_dataset()
        trans_mat = dataset.transformation_matrix
        conv_latt = Lattice(np.transpose(np.dot(np.transpose(structure.lattice.matrix), np.linalg.inv(trans_mat))))
        xtal_sys = sga.get_crystal_system()

        vecs = conv_latt.matrix
        lengths = np.array(conv_latt.abc)
        angles = np.array(conv_latt.angles)
        rotation = np.zeros((3, 3))

        # IEEE rules: a,b,c || x1,x2,x3
        if xtal_sys == "cubic":
            rotation = [vecs[i] / lengths[i] for i in range(3)]

        # IEEE rules: a=b in length; c,a || x3, x1
        elif xtal_sys == "tetragonal":
            rotation = np.array(
                [vec / mag for (mag, vec) in sorted(zip(lengths, vecs, strict=True), key=lambda x: x[0])]
            )
            if abs(lengths[2] - lengths[1]) < abs(lengths[1] - lengths[0]):
                rotation[0], rotation[2] = rotation[2], rotation[0].copy()
            rotation[1] = get_uvec(np.cross(rotation[2], rotation[0]))

        # IEEE rules: c<a<b; c,a || x3,x1
        elif xtal_sys == "orthorhombic":
            try:
                rotation = [vec / mag for (mag, vec) in sorted(zip(lengths, vecs, strict=True))]
            except ValueError:
                rotation = [vec / mag for (mag, vec) in sorted(zip(lengths, vecs, strict=True), key=lambda x: x[0])]
            rotation = np.roll(rotation, 2, axis=0)

        # IEEE rules: c,a || x3,x1, c is threefold axis
        # Note this also includes rhombohedral crystal systems
        elif xtal_sys in ("trigonal", "hexagonal"):
            # find threefold axis:
            tf_index = np.argmin(abs(angles - 120.0))
            non_tf_mask = np.logical_not(angles == angles[tf_index])
            rotation[2] = get_uvec(vecs[tf_index])
            rotation[0] = get_uvec(vecs[non_tf_mask][0])
            rotation[1] = get_uvec(np.cross(rotation[2], rotation[0]))

        # IEEE rules: b,c || x2,x3; alpha=beta=90, c<a
        elif xtal_sys == "monoclinic":
            # Find unique axis
            u_index = np.argmax(abs(angles - 90.0))
            n_umask = np.logical_not(angles == angles[u_index])
            rotation[1] = get_uvec(vecs[u_index])
            # Shorter of remaining lattice vectors for c axis
            c = next(vec / mag for (mag, vec) in sorted(zip(lengths[n_umask], vecs[n_umask], strict=True)))
            rotation[2] = np.array(c)
            rotation[0] = np.cross(rotation[1], rotation[2])

        # IEEE rules: c || x3, x2 normal to ac plane
        elif xtal_sys == "triclinic":
            try:
                rotation = [vec / mag for (mag, vec) in sorted(zip(lengths, vecs, strict=True))]
            except ValueError:
                rotation = [vec / mag for (mag, vec) in sorted(zip(lengths, vecs, strict=True), key=lambda x: x[0])]
            rotation[1] = get_uvec(np.cross(rotation[2], rotation[0]))
            rotation[0] = np.cross(rotation[1], rotation[2])

        rotation = SquareTensor(rotation)
        if refine_rotation:
            rotation = rotation.refine_rotation()

        return rotation

In [273]:
R_z = np.array([ # rotation of +90 around z
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, 1],
])
R_x = np.array([ # rotation of +90 around x
    [1, 0, 0],
    [0, 0, -1],
    [0, 1, 0],
])
R_y = np.array([ # rotation of +90 around y
    [0, 0, 1],
    [0, 1, 0],
    [-1, 0, 0],
])

In [274]:
# tol_rel_null_full = 5e-2
tol_rel_null_full = 1e-1

import warnings
# warnings.simplefilter('error', UserWarning)

path_df_rot_ieee = "data/df_rot_ieee_pmg.pkl.gz"

if Path(path_df_rot_ieee).exists():
    df_rot_ieee = pd.read_pickle(path_df_rot_ieee)
else:
    list_dijk_rot               = []
    list_epsij_rot              = []
    list_structure_rot          = []
    list_rot_is_conventional    = []
    list_pg_match               = []
    
    for ir, r in df.iterrows():
        d = r['dijk']
        eps = r['epsij']
        structure = Structure.from_dict(r['structure'])
        pg = r['abi_pg_symbol']
        spg = r['abi_spg_symbol']

        spga = SpacegroupAnalyzer(structure=structure, symprec=0.001) # dflt 0.01
        if pg!=spga.get_point_group_symbol():
            list_pg_match.append(False)
        else:
            list_pg_match.append(True)
    
        if  verify_shg_conventional_shape_by_pg(d, pg_symbol=pg, tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=spg)[0]:
            d_rot = copy(d)
            eps_rot = copy(eps)
            structure_new = structure.copy()

        # not a CS per se but zero response in static limit by symmetry
        elif pg=="422":
            d_rot = np.zeros_like(np.array(d))
            eps_rot = copy(eps)
            structure_new = structure.copy()

        else:
            try:
                # d_rot = np.array(SHGTensor(np.array(d)).convert_to_ieee(structure=structure))
                # eps_rot = np.array(SHGTensor(np.array(eps)).convert_to_ieee(structure=structure))
                # if np.allclose(d_rot, 0.0):
                d_rot = np.array(SHGTensor(np.array(d)).convert_to_ieee(structure=structure, initial_fit=False))
                eps_rot = np.array(SHGTensor(np.array(eps)).convert_to_ieee(structure=structure, initial_fit=False))

                R_ieee = np.array(SHGTensor(np.array(d)).get_ieee_rotation(structure=structure))
                structure_new = structure.copy().apply_operation(SymmOp.from_rotation_and_translation(rotation_matrix=R_ieee))
                rot_pmg_ieee = True
            except StopIteration:
                d_rot = copy(d)
                eps_rot = copy(eps)
                structure_new = structure.copy()
                rot_pmg_ieee = False

            if pg=="mm2": # bc IEEE conventions not adapted for mm2 (see Roberts 1992)
                sga = SpacegroupAnalyzer(structure_new, symprec=0.001) # dflt 0.01
                for symmop in sga.get_point_group_operations(cartesian=True)[1:]: # [1:] to avoid identity
                    if math.isclose(np.prod(np.diag(symmop.affine_matrix[:3,:3])), 1): # only operation 2 is True
                        symmop_2 = symmop.affine_matrix[:3,:3]
                        symmop_2 = np.round(symmop_2) # to avoid dealing with 0.9999999999
                        break
                else:
                    raise Exception(f"Symmop 2 not found for pg mm2 in material {ir}")
                dataset = sga.get_symmetry_dataset()
                trans_mat = dataset.transformation_matrix
                conv_latt = Lattice(np.transpose(np.dot(np.transpose(structure_new.lattice.matrix), np.linalg.inv(trans_mat))))
                arg_polar_axis = np.where(symmop_2==1)[0][0]
                vec_polar_axis = conv_latt.matrix[arg_polar_axis]
                t = [0,1,2]
                t.remove(arg_polar_axis)
                vec0 = conv_latt.matrix[t,:][0]
                vec1 = conv_latt.matrix[t,:][1]
                if np.linalg.norm(vec0)<np.linalg.norm(vec1):
                    vec_a = vec0
                else:
                    vec_a = vec1

                while   not math.isclose(vec_polar_axis[2]/np.linalg.norm(vec_polar_axis), 1) and \
                        not math.isclose(vec_a[0]/np.linalg.norm(vec_a), 1):
                    if math.isclose(symmop_2[0,0], 1):
                        angles_for_mm2 = [0, 90, 0]
                        R_axis2 = R_y
                    elif math.isclose(symmop_2[1,1], 1):
                        angles_for_mm2 = [90, 0, 0]
                        R_axis2 = R_x
                    elif math.isclose(symmop_2[2,2], 1) and math.isclose(vec_polar_axis[2]/np.linalg.norm(vec_polar_axis), -1):
                        angles_for_mm2 = [180, 0, 0]
                        R_axis2 = np.array([ # rotation of +180 around x
                            [1, 0, 0],
                            [0, -1, 0],
                            [0, 0, -1],
                        ])
                    elif math.isclose(symmop_2[2,2], 1) and math.isclose(vec_polar_axis[2]/np.linalg.norm(vec_polar_axis), 1):
                        angles_for_mm2 = [0, 0, 90]
                        R_axis2 = R_z
                    else:
                        raise Exception(f"Something is wrong with the symmop_2: {symmop_2} ")
                    d_rot = shg.apply_rot(d_rot, *angles_for_mm2)
                    eps_rot = shg.apply_rot(eps_rot, *angles_for_mm2)
                    structure_new = structure_new.copy().apply_operation(SymmOp.from_rotation_and_translation(rotation_matrix=R_axis2))

                    sga = SpacegroupAnalyzer(structure_new, symprec=0.001) # dflt 0.01
                    for symmop in sga.get_point_group_operations(cartesian=True)[1:]: # [1:] to avoid identity
                        if math.isclose(np.prod(np.diag(symmop.affine_matrix[:3,:3])), 1): # only operation 2 is True
                            symmop_2 = symmop.affine_matrix[:3,:3]
                            symmop_2 = np.round(symmop_2)
                            break
                    arg_polar_axis = np.where(symmop_2==1)[0][0]
                    dataset = sga.get_symmetry_dataset()
                    trans_mat = dataset.transformation_matrix
                    conv_latt = Lattice(np.transpose(np.dot(np.transpose(structure_new.lattice.matrix), np.linalg.inv(trans_mat))))
                    arg_polar_axis = np.where(symmop_2==1)[0][0]
                    vec_polar_axis = conv_latt.matrix[arg_polar_axis]
                    t = [0,1,2]
                    t.remove(arg_polar_axis)
                    vec0 = conv_latt.matrix[t,:][0]
                    vec1 = conv_latt.matrix[t,:][1]
                    if np.linalg.norm(vec0)<np.linalg.norm(vec1):
                        vec_a = vec0
                    else:
                        vec_a = vec1
        
            # Align mirror plan with x-z plan (-1 at symmop[1,1]), if b//-y, then rotate by 180 around x, 
            # then align the smallest lattice parameter (conventional setting for all this) with z
            # Needed bc some monoclinic are highly symmetrical with three conventional angles of 90 degrees 
            # such that the algorithm throws errors
            elif pg=="m" and not rot_pmg_ieee:
                sga = SpacegroupAnalyzer(structure_new, symprec=0.001) # dflt 0.01
                for symmop in sga.get_point_group_operations(cartesian=True)[1:]: # [1:] to avoid identity
                    symmop_m = np.round(symmop.affine_matrix[:3,:3])
                    if -1 in np.diag(symmop_m) and 1 in np.diag(symmop_m) and np.prod(np.diag(symmop_m))==-1:
                        break
                dataset = sga.get_symmetry_dataset()
                trans_mat = dataset.transformation_matrix
                conv_latt = Lattice(np.transpose(np.dot(np.transpose(structure_new.lattice.matrix), np.linalg.inv(trans_mat))))
                arg_axis_normal_m = np.where(symmop_m==-1)[0][0]
                vec_axis_normal_m = conv_latt.matrix[arg_axis_normal_m]
                t = [0,1,2]
                t.remove(arg_axis_normal_m)
                vec0 = conv_latt.matrix[t,:][0]
                vec1 = conv_latt.matrix[t,:][1]
                if np.linalg.norm(vec0)<np.linalg.norm(vec1):
                    vec_c = vec0
                else:
                    vec_c = vec1
        
                while   not math.isclose(vec_axis_normal_m[1]/np.linalg.norm(vec_axis_normal_m), 1) and \
                        not math.isclose(vec_c[2]/np.linalg.norm(vec_c), 1):
                    if math.isclose(symmop_m[0,0], -1):
                        angles_for_m = [0, 0, 90]
                        R_axis2 = R_z
                    elif math.isclose(symmop_m[1,1], -1) and math.isclose(vec_axis_normal_m[1]/np.linalg.norm(vec_axis_normal_m), -1):
                        angles_for_m = [180, 0, 0]
                        R_axis2 = np.array([ # rotation of +180 around x
                            [1, 0, 0],
                            [0, -1, 0],
                            [0, 0, -1],
                        ])
                    elif math.isclose(symmop_m[1,1], -1) and math.isclose(vec_axis_normal_m[1]/np.linalg.norm(vec_axis_normal_m), 1):
                        angles_for_m = [0, 90, 0]
                        R_axis2 = R_y
                    elif math.isclose(symmop_m[2,2], -1):
                        angles_for_m = [90, 0, 0]
                        R_axis2 = R_x
                    else:
                        raise Exception(f"Something is wrong with the symmop_m: {symmop_m} ")
                    d_rot = shg.apply_rot(d_rot, *angles_for_m)
                    eps_rot = shg.apply_rot(eps_rot, *angles_for_m)
                    structure_new = structure_new.copy().apply_operation(SymmOp.from_rotation_and_translation(rotation_matrix=R_axis2))
        
                    sga = SpacegroupAnalyzer(structure_new, symprec=0.001) # dflt 0.01
                    for symmop in sga.get_point_group_operations(cartesian=True)[1:]: # [1:] to avoid identity
                        symmop_m = np.round(symmop.affine_matrix[:3,:3])
                        if -1 in np.diag(symmop_m) and 1 in np.diag(symmop_m) and np.prod(np.diag(symmop_m))==-1:
                            break
                    dataset = sga.get_symmetry_dataset()
                    trans_mat = dataset.transformation_matrix
                    conv_latt = Lattice(np.transpose(np.dot(np.transpose(structure_new.lattice.matrix), np.linalg.inv(trans_mat))))
                    arg_axis_normal_m = np.where(symmop_m==-1)[0][0]
                    vec_axis_normal_m = conv_latt.matrix[arg_axis_normal_m]
                    t = [0,1,2]
                    t.remove(arg_axis_normal_m)
                    vec0 = conv_latt.matrix[t,:][0]
                    vec1 = conv_latt.matrix[t,:][1]
                    if np.linalg.norm(vec0)<np.linalg.norm(vec1):
                        vec_c = vec0
                    else:
                        vec_c = vec1


        list_dijk_rot.append(list(d_rot))
        list_epsij_rot.append(list(eps_rot))
        list_structure_rot.append(structure_new.as_dict())
        if pg == "422":
            list_rot_is_conventional.append(True)
        elif not verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=pg, tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=spg)[0]:
            list_rot_is_conventional.append(False)
        else:
            list_rot_is_conventional.append(True)

    df_rot_ieee = df.copy()

    df_rot_ieee['pg_match_abi_spga'] = list_pg_match
    df_rot_ieee['dijk_rot'] = list_dijk_rot
    df_rot_ieee['epsij_rot'] = list_epsij_rot
    df_rot_ieee['structure_rot'] = list_structure_rot
    df_rot_ieee['rot_is_conventional'] = list_rot_is_conventional

    # for ir, r in df_rot_ieee[(df_rot_ieee['abi_pg_symbol'].isin(["mm2", "-6m2", "P31m"])) & (df_rot_ieee['rot_is_conventional']==False)].iterrows():
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        for angles in [[90, 0, 0], [0, 90, 0], [0, 0, 90], [90, 90, 0], [90, 0, 90], [0, 90, 90], [90, 90, 90]]:
            d_rot = shg.apply_rot(r['dijk_rot'], *angles)
            eps_rot = shg.apply_rot(r['epsij_rot'], *angles)
            structure = Structure.from_dict(r['structure_rot'])
            structure_new = structure.copy()
            structure_new.lattice = Lattice(matrix=shg.apply_rot_syst(structure.lattice.matrix, *angles))
            if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=r['abi_pg_symbol'], tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=r['abi_spg_symbol'])[0]:
                df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
                df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
                df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
                df_rot_ieee.at[ir, 'rot_is_conventional'] = True
                break
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        for angles in [[90, 0, 0], [0, 90, 0], [0, 0, 90], [90, 90, 0], [90, 0, 90], [0, 90, 90], [90, 90, 90]]:
            d_rot = shg.apply_rot(r['dijk'], *angles)
            eps_rot = shg.apply_rot(r['epsij'], *angles)
            structure = Structure.from_dict(r['structure'])
            structure_new = structure.copy()
            structure_new.lattice = Lattice(matrix=shg.apply_rot_syst(structure.lattice.matrix, *angles))
            if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=r['abi_pg_symbol'], tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=r['abi_spg_symbol'])[0]:
                df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
                df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
                df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
                df_rot_ieee.at[ir, 'rot_is_conventional'] = True
                break


    for ir, r in df_rot_ieee[(df_rot_ieee['abi_pg_symbol']=="-42m") & (df_rot_ieee['rot_is_conventional']==False)].iterrows():
        angles = [0, 0, 45]
        d_rot = shg.apply_rot(r['dijk'], *angles)
        eps_rot = shg.apply_rot(r['epsij'], *angles)
        structure = Structure.from_dict(r['structure'])
        structure_new = structure.copy()
        structure_new.lattice = Lattice(matrix=shg.apply_rot_syst(structure.lattice.matrix, *angles))
        if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=r['abi_pg_symbol'], tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                spg_symbol=r['abi_spg_symbol'])[0]:
            df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
            df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
            df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
            df_rot_ieee.at[ir, 'rot_is_conventional'] = True



    df_rot_ieee['dKP_full_wrt_dKP'] = [0.0]*len(df_rot_ieee)
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        pg = r['abi_pg_symbol']
        spg = r['abi_spg_symbol']
        dKP_start = shg.get_dKP(r['dijk'])
        dKP_full = shg.get_dKP(get_full_d(r['dijk'], pg_symbol=pg, spg_symbol=spg))
        if verify_shg_conventional_shape_by_pg(np.array(r['dijk']), pg_symbol=pg, spg_symbol=spg, tol_rel_null_full=1.5)[0] and \
        np.abs(dKP_start-dKP_full)/dKP_start <= 0.1:
            df_rot_ieee.at[ir, 'dijk_rot'] = list(r['dijk'])
            df_rot_ieee.at[ir, 'epsij_rot'] = list(r['epsij'])
            df_rot_ieee.at[ir, 'structure_rot'] = r['structure']
            df_rot_ieee.at[ir, 'rot_is_conventional'] = True
            df_rot_ieee.at[ir, 'dKP_full_wrt_dKP'] = np.abs(dKP_start-dKP_full)/dKP_start
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        pg = r['abi_pg_symbol']
        spg = r['abi_spg_symbol']
        dKP_start = shg.get_dKP(r['dijk_rot'])
        dKP_full = shg.get_dKP(get_full_d(r['dijk_rot'], pg_symbol=pg, spg_symbol=spg))
        if verify_shg_conventional_shape_by_pg(np.array(r['dijk_rot']), pg_symbol=pg, spg_symbol=spg, tol_rel_null_full=1.50)[0] and \
        np.abs(dKP_start-dKP_full)/dKP_start <= 0.1:
            df_rot_ieee.at[ir, 'rot_is_conventional'] = True
            df_rot_ieee.at[ir, 'dKP_full_wrt_dKP'] = np.abs(dKP_start-dKP_full)/dKP_start



    df_rot_ieee['spga_001_spg_for_conventional'] = ["abi_spg_ok_when_conventional"]*len(df_rot_ieee)
    df_rot_ieee['spga_001_pg_for_conventional'] = ["abi_pg_ok_when_conventional"]*len(df_rot_ieee)
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        for angles in [[0, 0, 0], [90, 0, 0], [0, 90, 0], [0, 0, 90], [90, 90, 0], [90, 0, 90], [0, 90, 90], [90, 90, 90]]:
            d_rot = shg.apply_rot(r['dijk_rot'], *angles)
            eps_rot = shg.apply_rot(r['epsij_rot'], *angles)
            structure = Structure.from_dict(r['structure_rot'])
            structure_new = structure.copy()
            structure_new.lattice = Lattice(matrix=shg.apply_rot_syst(structure.lattice.matrix, *angles))
            spga = SpacegroupAnalyzer(structure=structure, symprec=0.01)
            if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=spga.get_point_group_symbol(), tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=spga.get_space_group_symbol())[0]:
                df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
                df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
                df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
                df_rot_ieee.at[ir, 'rot_is_conventional'] = True
                df_rot_ieee.at[ir, 'spga_001_pg_for_conventional'] = spga.get_point_group_symbol()
                df_rot_ieee.at[ir, 'spga_001_spg_for_conventional'] = spga.get_space_group_symbol()
                break
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        for angles in [[0, 0, 0], [90, 0, 0], [0, 90, 0], [0, 0, 90], [90, 90, 0], [90, 0, 90], [0, 90, 90], [90, 90, 90]]:
            d_rot = shg.apply_rot(r['dijk'], *angles)
            eps_rot = shg.apply_rot(r['epsij'], *angles)
            structure = Structure.from_dict(r['structure'])
            structure_new = structure.copy()
            structure_new.lattice = Lattice(matrix=shg.apply_rot_syst(structure.lattice.matrix, *angles))
            spga = SpacegroupAnalyzer(structure=structure, symprec=0.01)
            if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=spga.get_point_group_symbol(), tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=spga.get_space_group_symbol())[0]:
                df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
                df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
                df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
                df_rot_ieee.at[ir, 'rot_is_conventional'] = True
                df_rot_ieee.at[ir, 'spga_001_pg_for_conventional'] = spga.get_point_group_symbol()
                df_rot_ieee.at[ir, 'spga_001_spg_for_conventional'] = spga.get_space_group_symbol()
                break


    df_rot_ieee['rot_was_symmetrized'] = [False]*len(df_rot_ieee)
    # For the materials still not okay, let's allow a symmetrization of the tensor based on the structure given
    # BUT only if the new dKP is within 5% of the original one
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        try:
            d = r['dijk']
            structure = Structure.from_dict(r['structure'])
            dKP_start = shg.get_dKP(d)
            d_rot = np.array(SHGTensor(np.array(d)).convert_to_ieee(structure=structure, initial_fit=True))
            dKP_end = shg.get_dKP(d_rot)
            if np.abs(dKP_end-dKP_start)/dKP_start > 0.05:
                continue
            eps_rot = np.array(SHGTensor(np.array(eps)).convert_to_ieee(structure=structure, initial_fit=True))
            R_ieee = np.array(SHGTensor(np.array(d)).get_ieee_rotation(structure=structure))
            structure_new = structure.copy().apply_operation(SymmOp.from_rotation_and_translation(rotation_matrix=R_ieee))
            if verify_shg_conventional_shape_by_pg(d_rot, pg_symbol=r['abi_pg_symbol'], tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full, 
                                                    spg_symbol=r['abi_spg_symbol'])[0]:
                df_rot_ieee.at[ir, 'dijk_rot'] = list(d_rot)
                df_rot_ieee.at[ir, 'epsij_rot'] = list(eps_rot)
                df_rot_ieee.at[ir, 'structure_rot'] = structure_new.as_dict()
                df_rot_ieee.at[ir, 'rot_is_conventional'] = True
                df_rot_ieee.at[ir, 'rot_was_symmetrized'] = True
        except StopIteration:
            continue

    df_rot_ieee['why_not_conventional_original'] = ["is_conventional"]*len(df_rot_ieee)
    df_rot_ieee['why_not_conventional_rot'] = ["is_conventional"]*len(df_rot_ieee)
    for ir, r in df_rot_ieee[df_rot_ieee['rot_is_conventional']==False].iterrows():
        pg = r['abi_pg_symbol']
        spg = r['abi_spg_symbol']
        df_rot_ieee.at[ir, "why_not_conventional_original"] = verify_shg_conventional_shape_by_pg(np.array(r['dijk']), pg_symbol=pg, spg_symbol=spg, tol_rel_null_full=0.50)[1]
        df_rot_ieee.at[ir, "why_not_conventional_rot"] = verify_shg_conventional_shape_by_pg(np.array(r['dijk_rot']), pg_symbol=pg, spg_symbol=spg, tol_rel_null_full=0.50)[1]


    # recompute dKP
    list_dKP_rot = []
    for ir, r in df_rot_ieee.iterrows():
        list_dKP_rot.append(shg.get_dKP(r['dijk_rot']))
    df_rot_ieee['dKP_rot'] = list_dKP_rot

    df_rot_ieee.to_pickle(path_df_rot_ieee)

# print(len(df_rot_ieee))
# display(df_rot_ieee.head())

In [275]:
df_rot_ieee_not_conv = df_rot_ieee[df_rot_ieee["rot_is_conventional"]==False]
print(len(df_rot_ieee_not_conv))
# df_rot_ieee_not_conv = df_rot_ieee_not_conv[df_rot_ieee_not_conv["why_not_conventional_rot"]!="neumann"]
# print(len(df_rot_ieee_not_conv))
# df_rot_ieee_not_conv = df_rot_ieee_not_conv[df_rot_ieee_not_conv["why_not_conventional_original"]!="neumann"]
# print(len(df_rot_ieee_not_conv))
print(df_rot_ieee_not_conv['abi_pg_symbol'].value_counts())
print(df_rot_ieee_not_conv['abi_spg_symbol'].value_counts())

0
Series([], Name: abi_pg_symbol, dtype: int64)
Series([], Name: abi_spg_symbol, dtype: int64)


In [276]:
print(df_rot_ieee_not_conv['origin'].value_counts())

Series([], Name: origin, dtype: int64)


## Verification

In [277]:
tol_rel_null_full = 1e-1
for ir, r in df_rot_ieee.iterrows():
    d_rot = r['dijk_rot']
    if r['spga_001_spg_for_conventional']!="abi_spg_ok_when_conventional":
        pg = r['spga_001_pg_for_conventional']
        spg = r['spga_001_spg_for_conventional']
    else:
        pg = r['abi_pg_symbol']
        spg = r['abi_spg_symbol']

    if not verify_shg_conventional_shape_by_pg(
        np.array(d_rot),
        pg_symbol=pg, 
        spg_symbol=spg,
        tol_max_null=1e-6, tol_rel_null_full=tol_rel_null_full,
    ):
        print(ir)
    

## Restrict to "full" tensor

In [278]:
# Set the components that should be 0 in the conventional form to perfect 0
path_df_rot_ieee = "data/df_rot_ieee_pmg.pkl.gz"

if Path(path_df_rot_ieee).exists():
    df_rot_ieee = pd.read_pickle(path_df_rot_ieee)
else:
    list_dKP_full = []
    list_dijk_full = []
    for ir, r in df_rot_ieee.iterrows():
        d_rot = r['dijk_rot']
        if r['spga_001_spg_for_conventional']!="abi_spg_ok_when_conventional":
            pg = r['spga_001_pg_for_conventional']
            spg = r['spga_001_spg_for_conventional']
        else:
            pg = r['abi_pg_symbol']
            spg = r['abi_spg_symbol']
    
        list_dijk_full.append(get_full_d(r['dijk_rot'], pg_symbol=pg, spg_symbol=spg))
        list_dKP_full.append(shg.get_dKP(list_dijk_full[-1]))
    
    df_rot_ieee['dijk_full'] = list_dijk_full
    df_rot_ieee['dKP_full'] = list_dKP_full

    df_rot_ieee.to_pickle(path_df_rot_ieee)

In [279]:
import plotly.express as px
fig = px.scatter(
    x=np.arange(len(df_rot_ieee)),
    y=(df_rot_ieee['dKP']-df_rot_ieee['dKP_rot'])/df_rot_ieee['dKP'],
    text=df_rot_ieee.index
    )
fig.show()
fig = px.scatter(
    x=np.arange(len(df_rot_ieee)),
    y=(df_rot_ieee['dKP_full']-df_rot_ieee['dKP_rot'])/df_rot_ieee['dKP_rot'],
    text=df_rot_ieee.index
    )
fig.show()

## Force Neumannn

In [281]:
# Enforce the equality of certain components as demanded by Neumann's principle
path_df_rot_ieee = "data/df_rot_ieee_pmg.pkl.gz"

if not Path(path_df_rot_ieee).exists():
    df_rot_ieee = pd.read_pickle(path_df_rot_ieee)
else:
    list_dijk_full_neum = []
    list_dKP_full_neum = []
    for ir, r in df_rot_ieee.iterrows():
        d_rot = r['dijk_full']
        if r['spga_001_spg_for_conventional']!="abi_spg_ok_when_conventional":
            pg = r['spga_001_pg_for_conventional']
            spg = r['spga_001_spg_for_conventional']
        else:
            pg = r['abi_pg_symbol']
            spg = r['abi_spg_symbol']

        if not verify_shg_neumann_by_pg(d=d_rot, pg_symbol=pg, spg_symbol=spg, abs_tol=1e-9, rel_tol_coef=None):
            d_constr = correct_shg_neumann_by_pg(d_rot, pg_symbol=pg, spg_symbol=spg)
        else:
             d_constr = d_rot

        if   verify_shg_neumann_by_pg(d_constr, pg_symbol=pg, spg_symbol=spg, abs_tol=1e-9, rel_tol_coef=None) and\
            (verify_shg_conventional_shape_by_pg(d_constr, pg_symbol=pg, spg_symbol=spg,
                                                    tol_max_null=1e-9, tol_rel_null_full=1e-4)[0] or \
             pg=="422" # bc dijk=0 so impossible to verify shg_conventional_shape
            ):
            list_dijk_full_neum.append(list(shg.from_voigt(d_constr)))
            list_dKP_full_neum.append(shg.get_dKP(d_constr))
        else:
            raise Exception(f"{ir}")

    df_rot_ieee['dijk_full_neum'] = list_dijk_full_neum
    df_rot_ieee['dKP_full_neum'] = list_dKP_full_neum

    df_rot_ieee.to_pickle(path_df_rot_ieee)

In [282]:
import plotly.express as px
fig = px.scatter(
    x=np.arange(len(df_rot_ieee)),
    y=(df_rot_ieee['dKP_full_neum']-df_rot_ieee['dKP_full'])/df_rot_ieee['dKP_full'],
    text=df_rot_ieee.index
    )
fig.show()

# END